In [ ]:
import re
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

c:\Users\Javi\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Función para procesamiento de texto
def preprocess_text(text):
    text = text['text'].replace('\n', ' ') #Quitar saltos de línea
    text = re.sub(r'<.*?>', '', text)  # Eliminar etiquetas HTML
    text = re.sub(r'http\S+', '', text)  # Eliminar URLs
    text = re.sub(r'\[.*?\]', '', text)  # Eliminar referencias en formato [Texto]
    text =  re.sub(r'\(\d{8}\)', '', text) #Eliminar fechas entre parentesis
    text = re.sub(r'\s+', ' ', text).strip() #Quitar espacios extras
    text = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ.,;:()?!\s]', '', text) #Eliminar caracteres especiales
    return {'text': text}

# 📌 Carga del dataset
Se utiliza el dataset Wikimedia de Huggingface en español, específicamente la versión 20231101.es

In [ ]:
# Ruta a la carpeta que contiene los archivos .parquet
dataset = load_dataset("parquet", data_files="20231101.es/*.parquet", split="train")

# 📌 Preprocesamiento del dataset
Se utiliza la función de limpieza de los datos preprocess_text para quitar saltos de línea y limpiar carácteres que puedan venir con los artículos de Wikipedia.

In [ ]:
#Limpiar dataset
clean_dataset = dataset.map(preprocess_text)

# Filtrar registros donde 'text' no tenga contenido relevante
clean_dataset = clean_dataset.filter(lambda x: x['text'] is not None and len(x['text'].strip()) > 50)

# 📌 Definición del TextSplitter

Se selecciona el TextSplitter RecursiveCharacterTextSplitter con un tamaño de chunk de 1024 y un overlap de 150.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap  = 150
)

In [ ]:
# Aplicar chunking a cada texto individualmente
chunked_dataset = [text_splitter.split_text(text) for text in clean_dataset["text"]]

In [ ]:
# Aplanar la lista de listas en una sola lista
flattened_chunks = [chunk for sublist in chunked_dataset for chunk in sublist]

In [ ]:
# Convertir los chunks de texto a objetos Document de LangChain
documents = [Document(page_content=text) for text in flattened_chunks]

# 📌 Selección del Embedding

A continuación, se escoge el embedding de HuggingFace sentence-transformers. La selección de este embedding fue principalmente por su uso gratuito.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## Opción 1
La primera opción es cargar directamente los datos a la base de datos vectorial.

In [ ]:
vector_db = FAISS.from_documents(documents, embed_model)

## Opción 2
Debido a la gran cantidad de datos, la segunda opción fue ir cargando los datos por lotes, de modo que si ocurre algún error de memoria, no se pierda el progreso.

In [ ]:
batch_size = 500000  # Ajusta según tu RAM disponible
vector_db = None  # Inicializamos la base de datos
index_path = "vector_index.faiss"  # Archivo para guardar el índice


for i in range(440000, len(documents), batch_size):
    batch = documents[i : i + batch_size]  # Dividimos en lotes más pequeños
    print(f"Indexando documentos {i} a {i + len(batch)}...")

    if vector_db is None:
        vector_db = FAISS.from_documents(batch, embed_model)
    else:
        vector_db.add_documents(batch)

    # Guardar el índice FAISS
    vector_db.save_local("faiss_index")

print("Indexación completada!")

Indexando documentos 440000 a 940000...
Indexando documentos 940000 a 1440000...
Indexando documentos 1440000 a 1940000...
Indexando documentos 1940000 a 2440000...


: 

Finalmente, se guarda localmente los datos vectoriales para luego, ser utilizados por el modelo RAG.

In [ ]:
# Guardar el índice FAISS
vector_db.save_local("faiss_index")